In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_BASE = os.environ.get("OPENAI_API_BASE")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

LANGCHAIN_TRACING_V2 = os.environ.get("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = 'https://api.smith.langchain.com'
LANGCHAIN_API_KEY = os.environ.get("LANGCHAIN_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

llm = ChatOpenAI(model="qwen-max", temperature=0)

model_name = "C:\\Home\\Documents\\Projects\\models\\BAAI\\bge-large-en-v1.5"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embedding_model = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

c:\Apps\miniconda3\envs\rag\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Vector Store

In [3]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)

vectorstore = QdrantVectorStore(
    client=client,
    collection_name="rag_from_scratch",
    embedding=embedding_model,
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

# Generate HyDE document

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""

prompt_hyde = ChatPromptTemplate.from_template(template)

generate_docs_for_retrieval = (prompt_hyde | llm | StrOutputParser())

In [5]:
# Run
question = "What is task decomposition for LLM agents?"

hypothetical_doc = generate_docs_for_retrieval.invoke({"question": question})

print(hypothetical_doc)

**Task Decomposition for Large Language Model (LLM) Agents: A Framework for Enhanced Problem Solving**

In the realm of artificial intelligence, large language models (LLMs) have emerged as powerful tools capable of handling a wide array of tasks, from natural language understanding and generation to complex problem-solving. However, the effectiveness of LLMs in solving intricate and multifaceted problems can be significantly enhanced through the process of task decomposition. Task decomposition refers to the systematic breakdown of a complex task into smaller, more manageable sub-tasks, each of which can be addressed independently or in a coordinated manner. This approach not only simplifies the problem-solving process but also allows LLMs to leverage their strengths more effectively.

The primary goal of task decomposition is to transform a high-level, often ambiguous, task into a series of well-defined, discrete steps. This is particularly important for LLMs, as they are typically t

# Retrieval Chain

In [6]:
retrieval_chain = generate_docs_for_retrieval | retriever 

retireved_docs = retrieval_chain.invoke({"question": question})

retireved_docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', '_id': 'ca53f36c-9331-4f43-bc8f-5b5e8f788d57', '_collection_name': 'rag_from_scratch'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts p

# Final RAG Chain

In [7]:
# RAG

template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

result = final_rag_chain.invoke({"context": retireved_docs, "question": question})

print(result)

Task decomposition for LLM (Large Language Model) agents is the process of breaking down a complex task into smaller, more manageable subtasks. This technique helps the model to handle and solve intricate problems by addressing them step-by-step. The document mentions two key methods for task decomposition:

1. **Chain of Thought (CoT)**: This method, introduced by Wei et al. in 2022, involves instructing the model to think through the problem step by step. By doing so, the model can utilize more computational resources at test time to decompose difficult tasks into simpler, smaller steps. This not only makes the task easier to manage but also provides insight into the model's reasoning process.

2. **Tree of Thoughts**: Proposed by Yao et al. in 2023, this approach extends the concept of CoT by exploring multiple reasoning possibilities at each step. It first breaks down the problem into multiple thought steps and then generates multiple thoughts per step, forming a tree-like structur